In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from obspy.signal.detrend import polynomial
import obspy

In [ ]:
#list the files
filelist = os.listdir("rr_data_file")
#read them into pandas
df_list = [pd.read_csv(f'rr_data_file/{file}',delimiter = "\t",names=[f'{file}']) for file in filelist]
#print(df_list)

In [ ]:
min_len = min(map(len, df_list))
#print(min_len)
#to even the all txt files cropping them
small_df = pd.concat(df_list,axis=1)
big_df = (small_df.T).apply(pd.to_numeric, errors='coerce')
cropped_df = big_df.iloc[:,:min_len]
#print(cropped_df)

In [ ]:
#separating the dataframe into 2 batches
batch1 = cropped_df.iloc[:,:int(cropped_df.shape[1]/2)]
batch2 = cropped_df.iloc[:,int(cropped_df.shape[1]/2):]

In [ ]:
df_cum = batch1.cumsum(axis = 1, skipna = True)
N = batch1.shape[1]

#defining a detrending function to find delta
def detrend(df, s_vals, p_order):
    for j in range(len(df)):
      row = df.iloc[j,:]
      delta_S_B=np.zeros((len(s_vals),N))
      for idx, s in enumerate(s_vals):
        print(idx,s)
        intval = int(np.floor(N/(2*s)))
        for i in range(intval):
          #print(row[i*2*s:(i+1)*2*s])
          polynomial(row[i*2*s:(i+1)*2*s],p_order, plot =False)
        polynomial(row[intval*2*s:],p_order, plot =False)
        print(row[i+s])
        delta_S_B[idx,:]=np.array([row[i+s]-row[i] if i+s< N else row[i] for i in range(N)])
        #delta_S_B[intval*2*s:]=row[intval*2*s:]
    return delta_S_B

delta_S_B = detrend(df_cum[0:3],[1000], 3)
#print(delta_S_B)
#plt.plot(np.linspace(0,N,N), delta_S_B[0,:])
#counts, edges=np.histogram(delta_S_B[0,:], bins=int(np.sqrt(delta_S_B[0,:].size)), range=(-10, 10))
#bin_cen=(edges[1:]+edges[:-1])/2
#plt.scatter(bin_cen, counts, s=1)